# Imports

In [1]:
import os
from os import listdir
from os.path import isfile, join

import requests
from PIL import Image

import pandas as pd

# Source Files

In [2]:
# Source Params

cwd_path = os.getcwd()
rel_path = '../raw_data/Yoga-82/yoga_dataset_links'
my_path = os.path.join(cwd_path, rel_path)


files = [f'{my_path}/{f}' for f in listdir(my_path) if f.endswith('.txt')]

In [3]:
my_path

'/home/lscr/code/2023-q1-wagon/2023-q1-projects/YOGi/notebooks/../raw_data/Yoga-82/yoga_dataset_links'

# Load Master CSV

In [5]:
df = pd.read_csv('../raw_data/images/yoga-82_scraping.csv')
df.shape

(28450, 4)

,class,image_name,url,success
0,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...,657.jpg,http://lesliesaglio.com/wp-content/uploads/201...,0
1,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...,643.jpg,https://s-media-cache-ak0.pinimg.com/736x/36/a...,1
2,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...,640.jpg,http://static.squarespace.com/static/5387efd9e...,1
3,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...,623.jpg,https://www.melissawest.com/wp-content/uploads...,1
4,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...,615.jpg,https://www.melissawest.com/wp-content/uploads...,1


# Checking Each Image that has been downloaded

In [62]:
from PIL import Image

In [66]:
try:
    im = Image.open(f'{abs_save_root_path}/Akarna_Dhanurasana/0_151.jpg')
except IOError:
    print('Broken')

Broken


In [64]:
im.format

'WEBP'

In [65]:
im.show()

In [8]:
image_rel_root_path = '../raw_data/images/'
image_root_path = os.path.join(cwd_path, image_rel_root_path)

image_root_path

'/home/lscr/code/2023-q1-wagon/2023-q1-projects/YOGi/notebooks/../raw_data/images/'

In [16]:
# get all files within subfolders

for pose_folder in listdir(image_root_path)[:3]:
    folder_path = os.path.join(image_root_path, pose_folder)
    print(folder_path)
    # files_in_folder = [f'{pose_folder}/{f}' for f in listdir(os.path.join()) if f.endswith('.txt')]

/home/lscr/code/2023-q1-wagon/2023-q1-projects/YOGi/notebooks/../raw_data/images/Feathered_Peacock_Pose_or_Pincha_Mayurasana_
/home/lscr/code/2023-q1-wagon/2023-q1-projects/YOGi/notebooks/../raw_data/images/Noose_Pose_or_Pasasana_
/home/lscr/code/2023-q1-wagon/2023-q1-projects/YOGi/notebooks/../raw_data/images/Supported_Headstand_pose_or_Salamba_Sirsasana_
